In [1]:
from torchvision import transforms,datasets

train_transform = transforms.Compose([
    transforms.Resize(72),                                      # Slightly larger to allow cropping
    transforms.RandomCrop(64),                                  # Random 64x64 crop for augmentation
    transforms.RandomHorizontalFlip(p=0.5),                     # Flip with 50% probability
    transforms.RandomRotation(15),                              # Small rotation ±15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Color/brightness variation
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2),  # Blur sometimes
    transforms.ToTensor(),                                      # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))     # Normalize to mean=0.5, std=0.5
])

test_transform = transforms.Compose([
    transforms.Resize((64, 64)),                                # Resize to 64x64 for testing
    transforms.ToTensor(),                                      # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))     # Normalize with same mean & std
])

In [2]:
train_data = datasets.ImageFolder("PlantDataset/train", transform=train_transform)
test_data = datasets.ImageFolder("PlantDataset/test", transform=test_transform)

In [3]:
from torch.utils.data import DataLoader
batch_size = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True)


In [4]:
num_classes = len(train_data.classes)


In [5]:
import torch
from tuningmodel import CustomCNN
from trainNN import train

def objective(trial):
    # 1. Suggest hyperparameters
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout_conv = trial.suggest_float("dropout_conv", 0.2, 0.5)
    dropout_fc = trial.suggest_float("dropout_fc", 0.3, 0.6)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    # 2. Setup device and data
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 3. Build model and optimizer
    model = CustomCNN(num_classes, dropout_conv, dropout_fc).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = torch.nn.CrossEntropyLoss()

    # 4. Optional: Use a scheduler (or not)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    # 5. Train model using your function
    results = train(
        model=model,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=5,  # keep small for tuning
        device=device,
        scheduler=scheduler
    )

    # 6. Evaluate final validation accuracy
    final_test_acc = results["test_acc"][-1]
    return final_test_acc  


In [6]:
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)

print("Best trial:")
best_trial = study.best_trial
print(f"  Test Accuracy: {best_trial.value:.4f}")
print(f"  Parameters: {best_trial.params}")

[I 2025-06-25 19:03:53,979] A new study created in memory with name: no-name-62bd88b6-f2f2-4c13-afbf-76db255eaeb5


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.4474 | train_acc: 0.5372 | test_loss: 1.1162 | test_acc: 0.6339
Current LR: 0.000333
Epoch: 2 | train_loss: 0.9356 | train_acc: 0.6901 | test_loss: 0.8454 | test_acc: 0.7337
Current LR: 0.000333
Epoch: 3 | train_loss: 0.7446 | train_acc: 0.7514 | test_loss: 0.6960 | test_acc: 0.7675
Current LR: 0.000333
Epoch: 4 | train_loss: 0.6221 | train_acc: 0.7904 | test_loss: 0.5388 | test_acc: 0.8252
Current LR: 0.000333


[I 2025-06-25 19:08:31,180] Trial 0 finished with value: 0.8379807692307693 and parameters: {'batch_size': 64, 'lr': 0.00033284692645719257, 'dropout_conv': 0.35928677834362466, 'dropout_fc': 0.3212433101667139, 'weight_decay': 6.567976548943132e-05}. Best is trial 0 with value: 0.8379807692307693.


Epoch: 5 | train_loss: 0.5397 | train_acc: 0.8170 | test_loss: 0.4616 | test_acc: 0.8380
Current LR: 0.000333


...

Epoch: 5 | train_loss: 0.4660 | train_acc: 0.8459 | test_loss: 0.2888 | test_acc: 0.9062
Current LR: 0.000791


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.3685 | train_acc: 0.5589 | test_loss: 0.9692 | test_acc: 0.6702
Current LR: 0.002905
Epoch: 2 | train_loss: 0.8935 | train_acc: 0.7012 | test_loss: 0.6866 | test_acc: 0.7760
Current LR: 0.002905
Epoch: 3 | train_loss: 0.9878 | train_acc: 0.6772 | test_loss: 1.4839 | test_acc: 0.7671
Current LR: 0.002905
Epoch: 4 | train_loss: 0.7383 | train_acc: 0.7556 | test_loss: 0.8258 | test_acc: 0.7182
Current LR: 0.002905


[I 2025-06-25 20:08:11,892] Trial 14 finished with value: 0.7074519230769231 and parameters: {'batch_size': 64, 'lr': 0.0029052833998977705, 'dropout_conv': 0.28516302782848235, 'dropout_fc': 0.35428841804916894, 'weight_decay': 1.8729633583778432e-05}. Best is trial 13 with value: 0.90625.


Epoch: 5 | train_loss: 0.7365 | train_acc: 0.7548 | test_loss: 0.9770 | test_acc: 0.7075
Current LR: 0.002905
Best trial:
  Test Accuracy: 0.9062
  Parameters: {'batch_size': 64, 'lr': 0.0007910337487851204, 'dropout_conv': 0.30095545731589424, 'dropout_fc': 0.30298593423275455, 'weight_decay': 1.3976550688783783e-05}
